### Состав команды:

- Поляков Александр (Aleksandr Polyakov)
- Востриков Алексей (Alexey Vostrikoff)
- Садыков Динар (Dinar_Sadykow)
- Федотов Андрей (Andrei Fedotov)
- Каштанкина Ксения
- Овчинников Алексей (Aleksey)
- Пьянков Алексей
- Лукошко Роберт (Robert)
- Поперечный Богдан (Bogdan2105)

# Описание данных

train - обучающая


test - по которой строится лидерборд


*.pckl - это полные файлы (запикленные датафреймы) train_sample.pckl.zip - первые 100000 строк трейна (запикленные датафреймы)


*.pckl - это pickle-файлы при открытии которых открываются в датафреймы


вас может удивить большое число данных, это сделано специально в реальном мире вам доступна куча данных, потому вам всегда надо решать какие из них важны а какие нет трейн и тест фактически продублированы CSV файлами в которых есть только часть столбцов (чтоб можно было считать кусочками) так же есть первые 100000 строк трейна запикленных отдельно


использовать все данные не всегда обязательно, не исключено наличие мусора который вам не понадобится

train_4_col_2.csv.zip - кусок TRAIN, который содержит id, category, subcategory (строки все)

test_3_col_2.csv.zip - кусок TEST, который содержит id, category, subcategory (строки все)

train_4_col_3.csv.zip - кусок TRAIN, который содержит id, fields (строки все)

test_3_col_3.csv.zip - кусок TEST, который содержит id, fields (строки все)

train_sample.pckl.zip - кусок TRAIN, который содержит все поля, но только первые 100000 строк

train_4_col.csv.zip - кусок TRAIN, который содержит id, name, description, price (строки все)

test_4_col.csv.zip - кусок TEST, который содержит id, name, description (строки все)

# Сделаем наивное предсказание основанное на медиане

- Возьмём медиану от всех цен тренировочного набора
- Заполним все предсказания этой медианой

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
train_cat_subcat = pd.read_csv('./hack/train_4_col_2.csv', sep='\t').set_index('Unnamed: 0')

In [4]:
train_cat_subcat.head()

,id,category,subcategory
Unnamed: 0,,,
0,3edeb34cf93f490ff760af85,9,914
1,c98febd50dad3cc0ffc86085,22,2202
2,ade01e13912a46a99134cc75,22,2204
3,ab3e6941c11304c1519aef75,22,2209
5,252452a91c944a22c276d995,22,2204


In [5]:
subsam = pd.read_csv('./hack/submit_Sample.csv', sep=',')

In [6]:
subsam.head()

,id,price
0,285ea2e9935ccdeb8378c6a5,0
1,adfb73820bbb831257df6e95,0
2,783025601c36202f633fc6a5,0
3,2f0cd2d2e15dc90afd847f95,0
4,5c23a37902855a20172845a5,0


In [7]:
test_cat_subcat = pd.read_csv('./hack/test_3_col_2.csv', sep='\t').set_index('Unnamed: 0')

In [8]:
test_cat_subcat.head()

,id,category,subcategory
Unnamed: 0,,,
4,285ea2e9935ccdeb8378c6a5,6,603
9,adfb73820bbb831257df6e95,2,203
15,783025601c36202f633fc6a5,1,116
19,2f0cd2d2e15dc90afd847f95,10,1009
20,5c23a37902855a20172845a5,11,1104


In [9]:
train_cat_subcat.shape

(1748890, 3)

In [10]:
train_desc_price = pd.read_csv('./hack/train_4_col.csv', sep='\t').set_index('Unnamed: 0')

In [11]:
train_cat_subcat = train_cat_subcat.merge(train_desc_price, on='id')

In [12]:
train_cat_subcat.head()

,id,category,subcategory,name,description,price
0,3edeb34cf93f490ff760af85,9,914,Сумка DG,NaN,199900
1,c98febd50dad3cc0ffc86085,22,2202,Комплект,8-12 лет,35000
2,ade01e13912a46a99134cc75,22,2204,Пальтишко демисезонное,"На девочку 1,5 г,состояние хорошее",30000
3,ab3e6941c11304c1519aef75,22,2209,Attipas,"Размер-135mm, euro-22.5.Прочная, мягкая, не ск...",80000
4,252452a91c944a22c276d995,22,2204,Жилет теплый,NaN,50000


In [13]:
def prepare_data(df_input):
    df_output = df_input.copy()
    df_output = pd.get_dummies(df_output, columns = ['category','subcategory'])
    df_output = df_output.drop(['name','description'], axis = 1)
    df_output['price'] = df_output['price']/100
    return df_output

In [14]:
train_cat_subcat = prepare_data(train_cat_subcat)

In [15]:
naiv_const= train_cat_subcat['price'].median()

In [16]:
naiv_const

800.0

In [17]:
subsam['price'] = naiv_const
subsam.head()

,id,price
0,285ea2e9935ccdeb8378c6a5,800.0
1,adfb73820bbb831257df6e95,800.0
2,783025601c36202f633fc6a5,800.0
3,2f0cd2d2e15dc90afd847f95,800.0
4,5c23a37902855a20172845a5,800.0


In [18]:
subsam.to_csv( "naive_median.csv", sep=',', encoding='utf-8',index=False )

##### Результат 5.43309 очка

## Почему-то у результата в копейках очков больше чем у результата в рублях 

In [81]:
dft[dft['sample']=='train']['price'].median()

80000.0

In [82]:
subsam['price'] = dft[dft['sample']=='train']['price'].median()
subsam.to_csv( "median-2.csv", sep=',', encoding='utf-8',index=False )

### Результат 2.66799

# Обучим ближайших соседей на выборке из первых 100 000 строк файла

In [47]:
import pickle
from sklearn.neighbors import KNeighborsRegressor

In [52]:
df_train_100000 = pickle.load(open('/Users/aleksandrpolakov/!Обучение/Практический Machine Learning/!ХАКАТОН/hack/train_sample.pckl','rb'))



In [53]:
df_train_100000['sample'] = 'train'

In [54]:
test_cat_subcat = pd.read_csv('./hack/test_3_col_2.csv', sep='\t').set_index('Unnamed: 0')
test_cat_subcat.head()

,id,category,subcategory
Unnamed: 0,,,
4,285ea2e9935ccdeb8378c6a5,6,603
9,adfb73820bbb831257df6e95,2,203
15,783025601c36202f633fc6a5,1,116
19,2f0cd2d2e15dc90afd847f95,10,1009
20,5c23a37902855a20172845a5,11,1104


In [55]:
df_train_100000 = df_train_100000.drop(['can_buy','can_promote','contacts_visible',
                                'description', 'id','date_created','description','mortgage_available','subway','delivery_available','payment_available',
                               'images','location','fields','name'], axis = 1)
df_train_100000.head()

,category,price,subcategory,sample
0,9,199900,914,train
1,22,35000,2202,train
2,22,30000,2204,train
3,22,80000,2209,train
5,22,50000,2204,train


In [58]:
df_train_100000['id'] = 'id'

In [59]:
test_cat_subcat['price'] = 0.0

In [56]:
test_cat_subcat['sample'] = 'test'

In [62]:
dft = pd.concat([df_train_100000,test_cat_subcat])
dft.head()

/Users/aleksandrpolakov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


,category,id,price,sample,subcategory
0,9,id,199900.0,train,914
1,22,id,35000.0,train,2202
2,22,id,30000.0,train,2204
3,22,id,80000.0,train,2209
5,22,id,50000.0,train,2204


In [65]:
def prepare_dft (dft):
    df_output = dft.copy()
    df_output = pd.get_dummies(df_output, columns = ['category','subcategory'])    
    return df_output

In [66]:
dft = prepare_dft (dft)
dft.shape

(849525, 274)

In [68]:
X_train = dft[dft['sample'] == 'train'].drop(['id','price','sample'], axis = 1)
y_train = dft[dft['sample'] == 'train']['price']
X_test = dft[dft['sample'] == 'test'].drop(['id','price','sample'], axis = 1)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(100000, 271)
(100000,)
(749525, 271)


In [69]:
knn5 = KNeighborsRegressor(n_neighbors=5, 
                          weights='uniform', 
                          metric='manhattan')
knn5.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='manhattan',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform')

In [70]:
y_preds = knn5.predict(X_test)

In [71]:
print('Я ЧУТЬ НЕ ПОМЕРЛА!!!')

Я ЧУТЬ НЕ ПОМЕРЛА!!!


In [76]:
subsam['price'] = y_preds
subsam.head()

,id,price
0,285ea2e9935ccdeb8378c6a5,448980.0
1,adfb73820bbb831257df6e95,320000.0
2,783025601c36202f633fc6a5,820000.0
3,2f0cd2d2e15dc90afd847f95,83000.0
4,5c23a37902855a20172845a5,139000.0


In [77]:
subsam.to_csv( "KNN5.csv", sep=',', encoding='utf-8',index=False )

In [57]:
train_cat_subcat.head()

,id,price,category_1,category_2,category_3,category_4,category_5,category_6,category_7,category_8,...,subcategory_2209,subcategory_2210,subcategory_2211,subcategory_2212,subcategory_2213,subcategory_2214,subcategory_2215,subcategory_2216,subcategory_10001,subcategory_10002
0,3edeb34cf93f490ff760af85,1999.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,c98febd50dad3cc0ffc86085,350.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ade01e13912a46a99134cc75,300.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ab3e6941c11304c1519aef75,800.0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,252452a91c944a22c276d995,500.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
X = train_cat_subcat.drop(['id', 'price'], axis = 1)

y = train_cat_subcat['price']

In [ ]:
knn5 = KNeighborsRegressor(n_neighbors=5, 
                          weights='uniform', 
                          metric='manhattan')
knn5.fit(X, y)